# Pinecone Vector Database Integration

**References:**

    - [LanChain Pinecone](https://python.langchain.com/v0.2/docs/integrations/vectorstores/pinecone/)
    - [PineconeVectorStore API](https://api.python.langchain.com/en/latest/vectorstores/langchain_pinecone.vectorstores.PineconeVectorStore.html)

In [2]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
from langchain_community.document_loaders import TextLoader
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_pinecone import PineconeVectorStore



/Users/tjosh/miniconda3/envs/druginsights/lib/python3.10/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
load_dotenv("../.env")

True

In [7]:
# prepare modules for document extraction
loader = TextLoader("../data/druginsights_prettify_test_2.txt")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

embeddings = AzureOpenAIEmbeddings(
    azure_deployment=os.environ["AZURE_OPENAI_EMBEDDING_DEPLOYMENT_NAME"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
)

docs

[Document(page_content="name='ACE inhibitors' diseases_and_usecases='Used in the management of diabetic nephropathy, hypertension, and proteinuria in children with nephritis.' warnings='ACE inhibitors can cause a very rapid fall in blood pressure in volume-depleted children; treatment should therefore be initiated with very low doses. Adverse effects such as apnoea, seizures, renal failure, and severe unpredictable hypotension are very common in the first month of life and it is therefore recommended that ACE inhibitors are avoided whenever possible, particularly in preterm neonates.' indications='Indicated for the management of hypertension and proteinuria in children with nephritis and for the treatment of diabetic nephropathy.' contraindications='Contraindicated in children known to have severe bilateral renal artery stenosis or severe stenosis of the artery supplying a single functioning kidney. Also contraindicated in children with known or suspected renovascular disease, unless t

In [6]:
# Prepare pinecone

PC = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
PC_INDEX_NAME = os.environ["PINECONE_INDEX_NAME"]
PC.list_indexes()

{'indexes': [{'dimension': 1536,
              'host': 'axum-druginsights-fmmnhqp.svc.aped-4627-b74a.pinecone.io',
              'metric': 'cosine',
              'name': 'axum-druginsights',
              'spec': {'serverless': {'cloud': 'aws', 'region': 'us-east-1'}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [8]:
# run this only once, if the index is not created

# PC.create_index(
#             name=PC_INDEX_NAME,
#             dimension=1536,
#             metric="cosine",
#             spec=ServerlessSpec(cloud="aws", region="us-east-1"),
#         )

In [10]:
# Adding documents to the vector database. Be careful, this will overwrite the previous documents.
# If you want to add more documents, you can use the .add_documents method instead.
# run only once for the same documents
# vectorstore = PineconeVectorStore.from_documents(docs, embeddings, index_name=PC_INDEX_NAME)

# Or, load an existing vector store
vectorstore = PineconeVectorStore(embedding=embeddings, index_name=PC_INDEX_NAME)


In [11]:
# let us test out the document search

query = "ACE inhibitors"
docs = vectorstore.similarity_search(query, k=3)
print(len(docs))
print(docs[0].page_content)

3
name='ACE inhibitors' diseases_and_usecases='Used in the management of diabetic nephropathy, hypertension, and proteinuria in children with nephritis.' warnings='ACE inhibitors can cause a very rapid fall in blood pressure in volume-depleted children; treatment should therefore be initiated with very low doses. Adverse effects such as apnoea, seizures, renal failure, and severe unpredictable hypotension are very common in the first month of life and it is therefore recommended that ACE inhibitors are avoided whenever possible, particularly in preterm neonates.' indications='Indicated for the management of hypertension and proteinuria in children with nephritis and for the treatment of diabetic nephropathy.' contraindications='Contraindicated in children known to have severe bilateral renal artery stenosis or severe stenosis of the artery supplying a single functioning kidney. Also contraindicated in children with known or suspected renovascular disease, unless the blood pressure cann